# Aggregate responses

Aggregate the synthetic person-level data to find the proportion who gave each response to each question.

## Set-up

### Packages and file paths

In [1]:
# Import required packages
from collections import defaultdict
from dataclasses import dataclass
import numpy as np
import os
import pandas as pd

In [2]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Stores paths to data and files'''
    survey = '../data/survey_data'
    synthetic_data = 'synthetic_data_raw.csv'
    aggregate = 'aggregate_responses.csv'


paths = Paths()

In [3]:
# Import functions defined elsewhere
import sys
sys.path.append('../')
from utilities.response_labels import create_response_label_dict
from create_and_process_data.functions import aggregate_proportions, results_by_school_and_group

### Import raw data

In [4]:
data = pd.read_csv(os.path.join(paths.survey, paths.synthetic_data))
data.head()

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,peer_talk_listen_lab,peer_talk_helpful_lab,peer_talk_if_lab,accept_peer_lab,year_group_lab,fsm_lab,sen_lab,ethnicity_lab,english_additional_lab,school_lab
0,4.0,2.0,6.0,3.0,2.0,1.0,2.0,1.0,2.0,5.0,...,Fully,Somewhat helpful,NaN,Not at all,Year 10,Non-FSM,Non-SEN,Ethnic minority,No,School E
1,1.0,2.0,1.0,3.0,3.0,2.0,3.0,8.0,4.0,2.0,...,NaN,NaN,NaN,Slightly,Year 10,NaN,Non-SEN,Ethnic minority,No,School D
2,2.0,3.0,4.0,NaN,1.0,1.0,1.0,1.0,5.0,4.0,...,NaN,NaN,Very comfortable,Not at all,Year 10,Non-FSM,Non-SEN,White British,No,School E
3,2.0,5.0,5.0,2.0,2.0,2.0,1.0,3.0,NaN,2.0,...,NaN,NaN,Uncomfortable,Mostly,NaN,Non-FSM,Non-SEN,White British,No,School G
4,5.0,3.0,4.0,1.0,1.0,3.0,3.0,2.0,5.0,2.0,...,Slightly,Somewhat helpful,NaN,Not at all,Year 8,Non-FSM,Non-SEN,White British,Yes,School B


## Find the proportion giving each response to each measure, within a given group

In [5]:
# Make list of columns that we want to count responses for
response_col = [col for col in data.columns if (
    col.endswith('_lab') and col not in [
        'school_lab', 'gender_lab', 'transgender_lab', 'sexual_orientation_lab',
        'neurodivergent_lab', 'birth_parent1_lab', 'birth_parent2_lab',
        'birth_you_lab', 'birth_you_age_lab', 'young_carer_lab',
        'care_experience_lab', 'year_group_lab', 'fsm_lab', 'sen_lab',
        'ethnicity_lab', 'english_additional_lab'])]

# Preview the list
response_col[0:10]

['autonomy_pressure_lab',
 'autonomy_express_lab',
 'autonomy_decide_lab',
 'autonomy_told_lab',
 'autonomy_myself_lab',
 'autonomy_choice_lab',
 'life_satisfaction_lab',
 'optimism_future_lab',
 'optimism_best_lab',
 'optimism_good_lab']

In [6]:
# Import dictionary which contains the response options for each question,
# for which we want to know the answers to
labels = create_response_label_dict()

# Add 'NaN': 'No response' to each of the dictionaries
# They are stored as dictionary of dictionaries, so we loop through and
# update each one
for key, value in labels.items():
    value.update({np.nan: 'No response'})

# Preview one of the dictionary items
labels['autonomy']

{1: '1 - Completely not true',
 2: '2',
 3: '3',
 4: '4',
 5: '5 - Completely true',
 nan: 'No response'}

In [7]:
# Create version where every question has count 0, to use when a school has no
# pupils of a particular subgroup (i.e. no-one in certain FSM/SEN/gender/year)
no_pupils = aggregate_proportions(
    data=data, response_col=response_col, labels=labels)
no_pupils[['count', 'percentage', 'n_responses']] = 0
no_pupils.head()

,cat,cat_lab,count,percentage,measure,n_responses
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",0,0,autonomy_pressure,0
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",0,0,autonomy_express,0
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",0,0,autonomy_decide,0
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",0,0,autonomy_told,0
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",0,0,autonomy_myself,0


In [8]:
# Find results of aggregation for each school and pupil group
result = results_by_school_and_group(
    data=data, agg_func=aggregate_proportions, no_pupils=no_pupils,
    response_col=response_col, labels=labels)

# Hide results where n<10
result.loc[result['n_responses'] < 10,
           ['count', 'percentage', 'n_responses']] = np.nan

# Preview head of dataframe
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[21, 27, 23, 24, 22, 11]","[16.40625, 21.09375, 17.96875, 18.75, 17.1875,...",autonomy_pressure,128.0,School A,All,All,All,All
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[27, 29, 21, 25, 26, 0]","[21.09375, 22.65625, 16.40625, 19.53125, 20.31...",autonomy_express,128.0,School A,All,All,All,All
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[24, 23, 17, 32, 21, 11]","[18.75, 17.96875, 13.28125, 25.0, 16.40625, 8....",autonomy_decide,128.0,School A,All,All,All,All
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[26, 22, 15, 31, 21, 13]","[20.3125, 17.1875, 11.71875, 24.21875, 16.4062...",autonomy_told,128.0,School A,All,All,All,All
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[32, 22, 21, 21, 19, 13]","[25.0, 17.1875, 16.40625, 16.40625, 14.84375, ...",autonomy_myself,128.0,School A,All,All,All,All


In [9]:
# Preview school where I set there to be no SEN pupils
# Should show as NaN, as count was 0 which is <10
result[(result['school_lab'] == 'School B') & (result['sen_lab'] == 'SEN')]

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",NaN,NaN,autonomy_pressure,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",NaN,NaN,autonomy_express,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",NaN,NaN,autonomy_decide,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",NaN,NaN,autonomy_told,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",NaN,NaN,autonomy_myself,NaN,School B,All,All,All,SEN
...,...,...,...,...,...,...,...,...,...,...,...
0,"[0, 1, nan]","[No, Yes, No response]",NaN,NaN,peer_talk,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, nan]","[Not at all, Slightly, Mostly, Fully, No respo...",NaN,NaN,peer_talk_listen,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, nan]","[Not helpful, Somewhat helpful, Very helpful, ...",NaN,NaN,peer_talk_helpful,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, nan]","[Very uncomfortable, Uncomfortable, Comfortabl...",NaN,NaN,peer_talk_if,NaN,School B,All,All,All,SEN


In [10]:
# Show example of how the branching question is handled.
# Look at the counts in each dataframe.
display(
    data
    .loc[data['school_lab'] == 'School A', ['home_talk', 'home_talk_helpful']]
    .value_counts(dropna=False)
    .sort_index()
    .reset_index())
display(result[result['measure'] == 'home_talk_helpful'].head(1))

,home_talk,home_talk_helpful,count
0,0.0,NaN,65
1,1.0,1.0,24
2,1.0,2.0,16
3,1.0,3.0,14
4,1.0,NaN,4
5,NaN,NaN,5


,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
0,"[1, 2, 3, nan]","[Not helpful, Somewhat helpful, Very helpful, ...","[24, 16, 14, 4]","[41.37931034482759, 27.586206896551722, 24.137...",home_talk_helpful,58.0,School A,All,All,All,All


## Add groups for each measure

This uses add_keys() like `1_create_synthetic_data.ipynb`.

In [11]:
# Initialise dictionary of groups
groups = defaultdict(str)

# Define function for adding multiple keys with the same value
def add_keys(value, keys):
    '''
    Add multiple keys with the same value to the dictionary
    Inputs:
    value: String which is the value for all the keys
    keys: Array with the keys
    '''
    groups.update(dict.fromkeys(keys, value))

add_keys('autonomy', [
    'autonomy_pressure',
    'autonomy_express',
    'autonomy_decide',
    'autonomy_told',
    'autonomy_myself',
    'autonomy_choice'])
add_keys('life_satisfaction', ['life_satisfaction'])
add_keys('optimism', [
    'optimism_future',
    'optimism_best',
    'optimism_good',
    'optimism_work'])
add_keys('wellbeing', [
    'wellbeing_optimistic',
    'wellbeing_useful',
    'wellbeing_relaxed',
    'wellbeing_problems',
    'wellbeing_thinking',
    'wellbeing_close',
    'wellbeing_mind'])
add_keys('esteem', [
    'esteem_satisfied',
    'esteem_qualities',
    'esteem_well',
    'esteem_value',
    'esteem_good'])
add_keys('stress', [
    'stress_control',
    'stress_overcome',
    'stress_confident',
    'stress_way'])
add_keys('appearance', ['appearance_happy', 'appearance_feel'])
add_keys('negative', [
    'negative_lonely',
    'negative_unhappy',
    'negative_like',
    'negative_cry',
    'negative_school',
    'negative_worry',
    'negative_sleep',
    'negative_wake',
    'negative_shy',
    'negative_scared'])
add_keys('lonely', ['lonely'])
add_keys('support', ['support_ways', 'support_look'])

add_keys('sleep', ['sleep'])
add_keys('physical', ['physical_days', 'physical_hours'])
add_keys('free_like', ['free_like'])
add_keys('media', ['media_hours'])
add_keys('places', [
    'places_freq',
    'places_barriers___1',
    'places_barriers___2',
    'places_barriers___3',
    'places_barriers___4',
    'places_barriers___5',
    'places_barriers___6',
    'places_barriers___7',
    'places_barriers___8',
    'places_barriers___9'])
add_keys('school_belong', ['school_belong'])
add_keys('staff_relationship', [
    'staff_interest', 'staff_believe', 'staff_best', 'staff_listen'])

add_keys('talk', [
    'staff_talk', 'staff_talk_listen', 'staff_talk_helpful', 'staff_talk_if',
    'home_talk', 'home_talk_listen', 'home_talk_helpful', 'home_talk_if',
    'peer_talk', 'peer_talk_listen', 'peer_talk_helpful', 'peer_talk_if'])
add_keys('accept', [
    'accept_staff', 'accept_home', 'accept_local', 'accept_peer'])

add_keys('home_relationship', [
    'home_interest', 'home_believe', 'home_best', 'home_listen'])
add_keys('home_happy', ['home_happy'])

add_keys('local_env', ['local_safe', 'local_support', 'local_trust',
                       'local_neighbours', 'local_places'])
add_keys('discrim', ['discrim_race', 'discrim_gender', 'discrim_orientation',
                     'discrim_disability', 'discrim_faith'])
add_keys('belong_local', ['belong_local'])
add_keys('wealth', ['wealth'])
add_keys('future', ['future_options', 'future_interest', 'future_support'])
add_keys('climate', ['climate'])
add_keys('social', [
    'social_along', 'social_time', 'social_support', 'social_hard'])
add_keys('bully', ['bully_physical', 'bully_other', 'bully_cyber'])

# Preview the dictionary
groups

defaultdict(str,
            {'autonomy_pressure': 'autonomy',
             'autonomy_express': 'autonomy',
             'autonomy_decide': 'autonomy',
             'autonomy_told': 'autonomy',
             'autonomy_myself': 'autonomy',
             'autonomy_choice': 'autonomy',
             'life_satisfaction': 'life_satisfaction',
             'optimism_future': 'optimism',
             'optimism_best': 'optimism',
             'optimism_good': 'optimism',
             'optimism_work': 'optimism',
             'wellbeing_optimistic': 'wellbeing',
             'wellbeing_useful': 'wellbeing',
             'wellbeing_relaxed': 'wellbeing',
             'wellbeing_problems': 'wellbeing',
             'wellbeing_thinking': 'wellbeing',
             'wellbeing_close': 'wellbeing',
             'wellbeing_mind': 'wellbeing',
             'esteem_satisfied': 'esteem',
             'esteem_qualities': 'esteem',
             'esteem_well': 'esteem',
             'esteem_value': 'esteem',
  

In [12]:
# Add groups and preview
result['group'] = result['measure'].map(groups)
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,group
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[21, 27, 23, 24, 22, 11]","[16.40625, 21.09375, 17.96875, 18.75, 17.1875,...",autonomy_pressure,128.0,School A,All,All,All,All,autonomy
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[27, 29, 21, 25, 26, 0]","[21.09375, 22.65625, 16.40625, 19.53125, 20.31...",autonomy_express,128.0,School A,All,All,All,All,autonomy
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[24, 23, 17, 32, 21, 11]","[18.75, 17.96875, 13.28125, 25.0, 16.40625, 8....",autonomy_decide,128.0,School A,All,All,All,All,autonomy
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[26, 22, 15, 31, 21, 13]","[20.3125, 17.1875, 11.71875, 24.21875, 16.4062...",autonomy_told,128.0,School A,All,All,All,All,autonomy
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[32, 22, 21, 21, 19, 13]","[25.0, 17.1875, 16.40625, 16.40625, 14.84375, ...",autonomy_myself,128.0,School A,All,All,All,All,autonomy


## Add labels for each measure

In [13]:
# Define labels
labels = {
    'autonomy_pressure': 'I feel pressured in my life',
    'autonomy_express': 'I generally feel free to express my ideas and opinions',
    'autonomy_decide': 'I feel like I am free to decide for myself how to live my life',
    'autonomy_told': 'In my daily life I often have to do what I am told',
    'autonomy_myself': 'I feel I can pretty much be myself in daily situations',
    'autonomy_choice': 'I have enough choice about how I spend my time',
    'life_satisfaction': 'Overall, how satisfied are you with your life nowadays?',
    'optimism_future': 'I am optimistic about my future',
    'optimism_best': 'In uncertain times, I expect the best',
    'optimism_good': 'I think good things are going to happen to me',
    'optimism_work': 'I believe that things will work out, no matter how difficult they seem',
    'wellbeing_optimistic': '''I've been feeling optimistic about the future''',
    'wellbeing_useful': '''I've been feeling useful''',
    'wellbeing_relaxed': '''I've been feeling relaxed''',
    'wellbeing_problems': '''I've been dealing with problems well''',
    'wellbeing_thinking': '''I've been thinking clearly''',
    'wellbeing_close': '''I've been feeling close to other people''',
    'wellbeing_mind': '''I've been able to make up my own mind about things''',
    'esteem_satisfied': 'On the whole, I am satisfied with myself',
    'esteem_qualities': 'I feel that I have a number of good qualities',
    'esteem_well': 'I am able to do things as well as most other people',
    'esteem_value': 'I am a person of value',
    'esteem_good': 'I feel good about myself',
    'stress_control': 'Felt you were unable to control the important things in your life',
    'stress_overcome': 'Felt that difficulties were piling up so high that you could not overcome them',
    'stress_confident': 'Felt confident about your ability to handle your personal problems',
    'stress_way': 'Felt that things were going your way',
    'appearance_happy': 'How happy are you with your appearance (the way that you look)?',
    'appearance_feel': 'My appearance affects how I feel about myself',
    'negative_lonely': 'I feel lonely',
    'negative_unhappy': 'I am unhappy',
    'negative_like': 'Nobody likes me',
    'negative_cry': 'I cry a lot',
    'negative_school': 'I worry when I am at school',
    'negative_worry': 'I worry a lot',
    'negative_sleep': 'I have problems sleeping',
    'negative_wake': 'I wake up in the night',
    'negative_shy': 'I am shy',
    'negative_scared': 'I feel scared',
    'lonely': 'How often do you feel lonely?',
    'support_ways': 'I have ways to support myself (e.g. to cope, or help myself feel better)',
    'support_look': 'I know where to look for advice on how to support myself',
    'sleep': 'Is the amount of sleep you normally get enough for you to feel awake and concentrate on your school work during the day?',
    'physical_days': 'How many days in a usual week are you physically active?',
    'physical_hours': 'How long on average do you spend being physically active?',
    'free_like': 'How often can you do things that you like in your free time?',
    'media_hours': 'On a normal weekday during term time, how much time do you spend on social media?',
    'places_freq': 'How many activities/places are there in your local area, that you choose to or would want to go to in your free time?',
    'places_barriers___1': '''There's nothing to do''',
    'places_barriers___2': '''I'm unable to get there and back''',
    'places_barriers___3': '''It's too expensive (to get there or take part)''',
    'places_barriers___4': 'Poor weather',
    'places_barriers___5': 'I have no-one to go with',
    'places_barriers___6': '''It's too busy''',
    'places_barriers___7': 'I feel uncomfortable/anxious about other people who might be there',
    'places_barriers___8': '''My parents/carers don't allow me to go''',
    'places_barriers___9': 'Other',
    'school_belong': 'I feel that I belong/belonged at my school',
    'staff_interest': 'At school there is an adult who... is interested in my schoolwork',
    'staff_believe': 'At school there is an adult who... believes that I will be a success',
    'staff_best': 'At school there is an adult who... wants me to do my best',
    'staff_listen': 'At school there is an adult who... listens to me when I have something to say',
    'staff_talk': 'Talked about feeling down with... an adult at school',
    'staff_talk_listen': 'Did you feel listened to when you spoke with... an adult at school',
    'staff_talk_helpful': 'Did you receive advice that you found helpful from... an adult at school',
    'staff_talk_if': 'How would you feel about speaking with... an adult at school',
    'accept_staff': 'Adults at your school',
    'home_interest': 'At home there is an adult who... is interested in my schoolwork',
    'home_believe': 'At home there is an adult who... believes that I will be a success',
    'home_best': 'At home there is an adult who... wants me to do my best',
    'home_listen': 'At home there is an adult who... listens to me when I have something to say',
    'home_talk': 'Talked about feeling down with... one of your parents/carers',
    'home_talk_listen': 'Did you feel listened to when you spoke with... one of your parents/carers',
    'home_talk_helpful': 'Did you receive advice that you found helpful from... one of your parents/carers',
    'home_talk_if': 'How would you feel about speaking with... one of your parents/carers',
    'accept_home': 'Your parents/carers',
    'home_happy': 'How happy are you with the home that you live in?',
    'local_safe': 'How safe do you feel when in your local area?',
    'local_support': 'People around here support each other with their wellbeing',
    'local_trust': 'You can trust people around here',
    'local_neighbours': 'I could ask for help or a favour from neighbours',
    'local_places': 'There are good places to spend your free time (e.g., leisure centres, parks, shops)',
    'discrim_race': 'How often do people make you feel bad because of... your race, skin colour or where you were born?',
    'discrim_gender': 'How often do people make you feel bad because of... your gender?',
    'discrim_orientation': 'How often do people make you feel bad because of... your sexual orientation?',
    'discrim_disability': 'How often do people make you feel bad because of... disability?',
    'discrim_faith': 'How often do people make you feel bad because of... your religion/faith?',
    'belong_local': 'I feel like I belong in my local area',
    'accept_local': 'People in your local area',
    'wealth': 'Compared to your friends, is your family richer, poorer or about the same?',
    'future_options': 'How many options are available?',
    'future_interest': 'How do you feel about the options available?',
    'future_support': 'Do you feel (or think you would feel) supported to explore options that interest you, even if no-one else around you has done them before?',
    'climate': 'How often do you worry about the impact of climate change on your future?',
    'social_along': 'I get along with people around me',
    'social_time': 'People like to spend time with me',
    'social_support': 'I feel supported by my friends',
    'social_hard': 'My friends care about me when times are hard (for example if I am sick or have done something wrong)',
    'bully_physical': 'How often do you get physically bullied at school? By this we mean getting hit, pushed around, threatened, or having belongings stolen.',
    'bully_other': 'How often do you get bullied in other ways at school? By this we mean insults, slurs, name calling, threats, getting left out or excluded by others, or having rumours spread about you on purpose.',
    'bully_cyber': 'How often do you get cyber-bullied? By this we mean someone sending mean text or online messages about you, creating a website making fun of you, posting pictures that make you look bad online, or sharing them with others.',
    'peer_talk': 'Talked about feeling down with... another person your age',
    'peer_talk_listen': 'Did you feel listened to when you spoke with... another person your age',
    'peer_talk_helpful': 'Did you receive advice that you found helpful from... another person your age',
    'peer_talk_if': 'How would you feel about speaking with... another person your age',
    'accept_peer': 'Other people your age'
}

In [14]:
# Add labels
result['measure_lab'] = result['measure'].map(labels)

# Preview
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,group,measure_lab
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[21, 27, 23, 24, 22, 11]","[16.40625, 21.09375, 17.96875, 18.75, 17.1875,...",autonomy_pressure,128.0,School A,All,All,All,All,autonomy,I feel pressured in my life
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[27, 29, 21, 25, 26, 0]","[21.09375, 22.65625, 16.40625, 19.53125, 20.31...",autonomy_express,128.0,School A,All,All,All,All,autonomy,I generally feel free to express my ideas and ...
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[24, 23, 17, 32, 21, 11]","[18.75, 17.96875, 13.28125, 25.0, 16.40625, 8....",autonomy_decide,128.0,School A,All,All,All,All,autonomy,I feel like I am free to decide for myself how...
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[26, 22, 15, 31, 21, 13]","[20.3125, 17.1875, 11.71875, 24.21875, 16.4062...",autonomy_told,128.0,School A,All,All,All,All,autonomy,In my daily life I often have to do what I am ...
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[32, 22, 21, 21, 19, 13]","[25.0, 17.1875, 16.40625, 16.40625, 14.84375, ...",autonomy_myself,128.0,School A,All,All,All,All,autonomy,I feel I can pretty much be myself in daily si...


## Save to csv

In [15]:
result.to_csv(os.path.join(paths.survey, paths.aggregate), index=False)